<a href="https://colab.research.google.com/github/vignesh99/CS6300-Speech-Technology/blob/master/HMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
                                    #Get data files onto root
%cd
from google.colab import drive
drive.mount('/content/gdrive')
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/1 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/2 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/3 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/6 /root
!cp -avr /content/gdrive/My\ Drive/HMM_IsoData/7 /root
!cp -avr /content/gdrive/My\ Drive/ctsHMM /root
!ls

/root
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
removed '/root/1'
'/content/gdrive/My Drive/HMM_IsoData/1' -> '/root/1'
removed '/root/2'
'/content/gdrive/My Drive/HMM_IsoData/2' -> '/root/2'
removed '/root/3'
'/content/gdrive/My Drive/HMM_IsoData/3' -> '/root/3'
removed '/root/6'
'/content/gdrive/My Drive/HMM_IsoData/6' -> '/root/6'
removed '/root/7'
'/content/gdrive/My Drive/HMM_IsoData/7' -> '/root/7'
'/content/gdrive/My Drive/ctsHMM/test/3.mfcc' -> '/root/ctsHMM/test/3.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/2.mfcc' -> '/root/ctsHMM/test/2.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/1.mfcc' -> '/root/ctsHMM/test/1.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/5.mfcc' -> '/root/ctsHMM/test/5.mfcc'
'/content/gdrive/My Drive/ctsHMM/test/4.mfcc' -> '/root/ctsHMM/test/4.mfcc'
'/content/gdrive/My Drive/ctsHMM/dev/277.mfcc' -> '/root/ctsHMM/dev/277.mfcc'
'/content/gdrive/My Drive/ctsHMM/dev/61.mfcc'

In [2]:
                                    #Test file
!cp -avr /root/1/dev/ra_1.mfcc /root
filex = "ra_1.mfcc"

'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'


In [3]:
                                    #Import required libraries
from pylab import *
import os                           #Going through folders
from scipy.cluster.vq import kmeans2
from google.colab import files
from sklearn.cluster import KMeans

**Discete HMMs**

In [4]:
def fileTofeature(fileMfcc) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
                                    
    return Nf,features


In [5]:
label,ft = fileTofeature(filex)
label

121

In [6]:
                                    #Give the file with sequences for training data
def trainCentre(Nclust=20,folders=None) :
    if folders is None :            #Go through all digits
        folders = array(["1","2","3","6","7"])

    ind = []                     #All digits total vector numbers
    ind.append([0,0])            #Inital values
    Nwin = []                       #No. of vectors in each train file
    count = 0
    for word in folders :
                                    #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "train":
                for f in files:
                    if f.endswith('.mfcc'):
                                    #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/train/$f /root
                        foo,temp = fileTofeature(f)
                                    #Concatenate training feature vectors
                        if count == 0 :
                            features = temp
                            Nwin.append(foo)
                        else :
                            features = np.concatenate((features,temp))
                            Nwin.append(foo+Nwin[-1])
                        !rm $f
                        count+=1
        ind.append([len(Nwin),Nwin[-1]])
                                    #Apply K-means on the vectors
    trainKM = KMeans(n_clusters=Nclust)
    trainKM.fit(features)
                                    #Get Nwin in required format
    ind = array(ind)
    Nwin = array(Nwin)
    for i in range(5) :
        Nwin[ind[i,0]:ind[i+1,0]] -=  ind[i,1]

    return Nwin,features,trainKM,ind


In [7]:
def seqDwnld(seq,fileWord) :
    with open(fileWord,"w") as f:
        f.write("\n".join(" ".join(map(str, line)) for line in seq))

    return None

In [8]:
                                    #Give the file with sequences for training data
def trainSeq(Nwin,features,trainKM,fileWord="check.txt") :
    seq = []
    count = 0
                                    #Go through required folders
    for i in range(0,len(Nwin)) :
                                    #Get each features from all-features-array
        if i != 0:
            temp = features[Nwin[i-1]:Nwin[i]]
        else : 
            temp = features[:Nwin[0]]
                                    #Perform Kmeans on predefined centres
        label = trainKM.predict(temp)
        seq.append(label)

        if i==2 :
            print(label)

    seq = array(seq)

    seqDwnld(seq,fileWord)

    return None


In [9]:
!ls

1  2  3  6  7  ctsHMM  ra_1.mfcc


In [10]:
Nw,feat,cenKM,ind = trainCentre()       #Train all the phenomes

                                    #Get file sequences
trainSeq(Nw[ind[0,0]:ind[1,0]],feat[ind[0,1]:ind[1,1]],cenKM,"one.seq")
trainSeq(Nw[ind[1,0]:ind[2,0]],feat[ind[1,1]:ind[2,1]],cenKM,"two.seq")
trainSeq(Nw[ind[2,0]:ind[3,0]],feat[ind[2,1]:ind[3,1]],cenKM,"three.seq")
trainSeq(Nw[ind[3,0]:ind[4,0]],feat[ind[3,1]:ind[4,1]],cenKM,"six.seq")
trainSeq(Nw[ind[4,0]:ind[5,0]],feat[ind[4,1]:ind[5,1]],cenKM,"seven.seq")


Directory: 1
Directory: train
'/root/1/train/ac_1.mfcc' -> '/root/ac_1.mfcc'
'/root/1/train/ag_1.mfcc' -> '/root/ag_1.mfcc'
'/root/1/train/ai_1.mfcc' -> '/root/ai_1.mfcc'
'/root/1/train/an_1.mfcc' -> '/root/an_1.mfcc'
'/root/1/train/bh_1.mfcc' -> '/root/bh_1.mfcc'
'/root/1/train/bi_1.mfcc' -> '/root/bi_1.mfcc'
'/root/1/train/br_1.mfcc' -> '/root/br_1.mfcc'
'/root/1/train/ca_1.mfcc' -> '/root/ca_1.mfcc'
'/root/1/train/cg_1.mfcc' -> '/root/cg_1.mfcc'
'/root/1/train/cl_1.mfcc' -> '/root/cl_1.mfcc'
'/root/1/train/cm_1.mfcc' -> '/root/cm_1.mfcc'
'/root/1/train/dc_1.mfcc' -> '/root/dc_1.mfcc'
'/root/1/train/dg_1.mfcc' -> '/root/dg_1.mfcc'
'/root/1/train/ea_1.mfcc' -> '/root/ea_1.mfcc'
'/root/1/train/ec_1.mfcc' -> '/root/ec_1.mfcc'
'/root/1/train/ee_1.mfcc' -> '/root/ee_1.mfcc'
'/root/1/train/eg_1.mfcc' -> '/root/eg_1.mfcc'
'/root/1/train/ei_1.mfcc' -> '/root/ei_1.mfcc'
'/root/1/train/ek_1.mfcc' -> '/root/ek_1.mfcc'
'/root/1/train/es_1.mfcc' -> '/root/es_1.mfcc'
'/root/1/train/hg_1.mfcc' -> '

In [15]:
                                    #Download the train sequence files
files.download("one.seq")
files.download("two.seq")
files.download("three.seq")
files.download("six.seq")
files.download("seven.seq")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
!ls
print(len(feat))
Nw

1  3  7       one.seq	 seven.seq  three.seq
2  6  ctsHMM  ra_1.mfcc  six.seq    two.seq
19968


array([  75,  159,  238,  322,  423,  520,  636,  730,  831,  925, 1028,
       1222, 1330, 1418, 1503, 1592, 1662, 1759, 1856, 1948, 2045, 2141,
       2238, 2359, 2444, 2538, 2646, 2721, 2818, 2910, 3034, 3146, 3224,
       3305, 3412, 3496, 3606, 3684, 3769,   70,  153,  233,  316,  422,
        525,  645,  743,  830,  937, 1038, 1126, 1205, 1289, 1372, 1444,
       1575, 1662, 1737, 1825, 1942, 2034, 2132, 2230, 2308, 2432, 2517,
       2601, 2677, 2770, 2876, 2969, 3068, 3153, 3256, 3343, 3456, 3534,
       3631,   71,  159,  242,  349,  452,  540,  646,  754,  843, 1024,
       1154, 1247, 1330, 1411, 1494, 1577, 1671, 1764, 1865, 1953, 2047,
       2159, 2275, 2458, 2538, 2621, 2733, 2829, 2930, 3024, 3148, 3254,
       3335, 3422, 3526, 3615, 3707, 3792, 3898,   97,  207,  341,  458,
        565,  696,  825,  946, 1061, 1169, 1309, 1435, 1548, 1645, 1761,
       1872, 1966, 2076, 2178, 2317, 2439, 2561, 2722, 2837, 2936, 3053,
       3161, 3269, 3377, 3490, 3635, 3760, 3876, 39

In [78]:
def testKmeans(fileMfcc,trainKM,Nclust=20,iters=1) :
                                    #Convert features from file into array
    features = np.genfromtxt(fileMfcc,delimiter='\n',dtype=str)
    Nc,Nf = features[0].split(" ")
    Nc = int(Nc)
    Nf = int(Nf)
    features = np.genfromtxt(features[1:],delimiter=' ',dtype=str)
    features = features.astype(float)
    
                                    #Apply K-means on the vectors
    # if len(features) == Nf and len(features[0]) == Nc :
    #     label = trainKM.predict(features)
    label = trainKM.predict(features)
    
    return label


In [13]:
                                    #Give the file with sequences for testing data
def testSeq(trainKM,fileWord="check.txt",folders=None) :
    if folders is None :
        folders = array(["1","2","3","6","7"])
    seq = []
    count = np.zeros(5)
    for word in folders :    
                                        #Go through required folders
        for subdir, dirs, files in os.walk(word):
            dirname = subdir.split(os.path.sep)[-1]
            print('Directory:', dirname)
            if dirname == "dev":
                for f in files:
                    if f.endswith(".mfcc"):
                                        #Copy file (mfcc) to root and perform K-means
                        !cp -avr /root/$word/dev/$f /root
                        label = testKmeans(f,trainKM)
                        seq.append(label)
                        !rm $f
                        count[np.where(word == folders)[0][0]] += 1
        if word != "1" :
            count[np.where(word == folders)[0][0]] = count[np.where(word == folders)[0][0]] + count[np.where(word == folders)[0][0]-1]

    print(count)
    seq = array(seq)
    
    seqDwnld(seq,fileWord)

    return count


In [14]:
                                    #Get the test sequence
count = testSeq(cenKM,"DHMM.test")

Directory: 1
Directory: train
Directory: dev
'/root/1/dev/mk_1.mfcc' -> '/root/mk_1.mfcc'
'/root/1/dev/mm_1.mfcc' -> '/root/mm_1.mfcc'
'/root/1/dev/ms_1.mfcc' -> '/root/ms_1.mfcc'
'/root/1/dev/mw_1.mfcc' -> '/root/mw_1.mfcc'
'/root/1/dev/nc_1.mfcc' -> '/root/nc_1.mfcc'
'/root/1/dev/ng_1.mfcc' -> '/root/ng_1.mfcc'
'/root/1/dev/nh_1.mfcc' -> '/root/nh_1.mfcc'
'/root/1/dev/pe_1.mfcc' -> '/root/pe_1.mfcc'
'/root/1/dev/pk_1.mfcc' -> '/root/pk_1.mfcc'
'/root/1/dev/pm_1.mfcc' -> '/root/pm_1.mfcc'
'/root/1/dev/pp_1.mfcc' -> '/root/pp_1.mfcc'
'/root/1/dev/ra_1.mfcc' -> '/root/ra_1.mfcc'
Directory: 2
Directory: train
Directory: dev
'/root/2/dev/mk_2.mfcc' -> '/root/mk_2.mfcc'
'/root/2/dev/mm_2.mfcc' -> '/root/mm_2.mfcc'
'/root/2/dev/ms_2.mfcc' -> '/root/ms_2.mfcc'
'/root/2/dev/mw_2.mfcc' -> '/root/mw_2.mfcc'
'/root/2/dev/nc_2.mfcc' -> '/root/nc_2.mfcc'
'/root/2/dev/ng_2.mfcc' -> '/root/ng_2.mfcc'
'/root/2/dev/nh_2.mfcc' -> '/root/nh_2.mfcc'
'/root/2/dev/pe_2.mfcc' -> '/root/pe_2.mfcc'
'/root/2/d

In [16]:
                                    #Download the test sequence files
files.download("DHMM.test")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
                                    #Final probability files
!cp -avr /content/gdrive/My\ Drive/fp1.txt /root
!cp -avr /content/gdrive/My\ Drive/fp2.txt /root
!cp -avr /content/gdrive/My\ Drive/fp3.txt /root
!cp -avr /content/gdrive/My\ Drive/fp6.txt /root
!cp -avr /content/gdrive/My\ Drive/fp7.txt /root
!ls

'/content/gdrive/My Drive/fp1.txt' -> '/root/fp1.txt'
'/content/gdrive/My Drive/fp2.txt' -> '/root/fp2.txt'
'/content/gdrive/My Drive/fp3.txt' -> '/root/fp3.txt'
'/content/gdrive/My Drive/fp6.txt' -> '/root/fp6.txt'
'/content/gdrive/My Drive/fp7.txt' -> '/root/fp7.txt'
1  3  7       DHMM.test  fp2.txt  fp6.txt  one.seq    six.seq	 two.seq
2  6  ctsHMM  fp1.txt	 fp3.txt  fp7.txt  seven.seq  three.seq


In [22]:
def testProb(fileProb) :
    prob = np.genfromtxt(fileProb,delimiter=' ',dtype=str)
    prob = prob.astype(float)

    return prob

In [23]:
def accModel(count) :

    error = np.zeros(5)
    prob1 = testProb("fp1.txt")
    allprob = np.zeros((len(prob1),5))
    allprob[:,0] = prob1
    
    allprob[:,1] = testProb("fp2.txt")
    allprob[:,2] = testProb("fp3.txt")
    allprob[:,3] = testProb("fp6.txt")
    allprob[:,4] = testProb("fp7.txt")
    
    estimate = np.argmax(allprob,axis=1)
    print(estimate)
    for i in range(0,5) :
        if i==0:
            error[i] = len(np.where(estimate[:count[i]]!=i)[0])
        else :
            error[i] = len(np.where(estimate[count[i-1]:count[i]]!=i)[0])

    print(error)
    acc = 1-(sum(error)/count[-1])
    
    return acc

In [24]:
acc = accModel(count.astype(int))
print(acc)

[0 0 2 0 0 0 0 0 0 0 0 0 1 1 2 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 1 4 4 4 4 4 4]
[1. 1. 0. 0. 1.]
0.95


**Continuous HMMs**

In [30]:

!cp -avr /content/gdrive/My\ Drive/one.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/two.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/three.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/six.seq.hmm /root
!cp -avr /content/gdrive/My\ Drive/seven.seq.hmm /root
#!cp -avr /content/gdrive/My\ Drive/12.hmm /root
!ls

'/content/gdrive/My Drive/one.seq.hmm' -> '/root/one.seq.hmm'
'/content/gdrive/My Drive/two.seq.hmm' -> '/root/two.seq.hmm'
'/content/gdrive/My Drive/three.seq.hmm' -> '/root/three.seq.hmm'
'/content/gdrive/My Drive/six.seq.hmm' -> '/root/six.seq.hmm'
'/content/gdrive/My Drive/seven.seq.hmm' -> '/root/seven.seq.hmm'
1	6	   DHMM.test  fp6.txt	   seven.seq	  three.seq
12.hmm	7	   fp1.txt    fp7.txt	   seven.seq.hmm  three.seq.hmm
2	ctsHMM	   fp2.txt    one.seq	   six.seq	  two.seq
3	ctsModels  fp3.txt    one.seq.hmm  six.seq.hmm	  two.seq.hmm


In [49]:
                                    #Concatanate 2-digit models
def filecombine(file1,file2,fileNew,folder="ctsModels",Nstate="6",p=0.9,check=[]) :
                                    #Read file1 - on top
    data1 = np.genfromtxt(file1,delimiter='\n',dtype=str)
                                    #Change the number of states
    headers = np.genfromtxt(data1[:2],delimiter=' ',dtype=str)
    headers[0,1] = Nstate
    headers = list(headers)
    headers.append("\n")
                                    #Read the initial probabilities
    probs1 = np.genfromtxt(data1[2:],delimiter='\t',dtype=str)
    #probs1 = probs1[:,:-1]
    probs1 = probs1.astype("float")
    probs1[-2][0] = 1-p
    probs1[-1][0] = p
                                    #Read file2 - bottom
    data2 = np.genfromtxt(file2,delimiter='\n',dtype=str)
                                    #Read the initial probabilities
    probs2 = np.genfromtxt(data2[2:],delimiter='\t',dtype=str)
    #probs2 = probs2[:,:-1]
    probs2 = probs2.astype("float")
    print(shape(probs1),shape(probs2))
                                    #Change the last state transition probabilities
    probs = np.concatenate((probs1,probs2))
    
                                    #Write to file
    with open(fileNew,"w") as f:
        f.write("\n".join(" ".join(map(str, line)) for line in headers))

    f = open(fileNew,'ab')
    np.savetxt(f,probs,delimiter='\t', fmt='%f')
    !mv $fileNew /root/$folder
    f.close()

    return None

In [45]:
def digit2model(files=None,folder="ctsModels") :
    if files is None :
        files = ["one.seq.hmm","two.seq.hmm","three.seq.hmm","six.seq.hmm","seven.seq.hmm"]
    opfiles = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
    opfiles = opfiles.astype("str")
    for i in range(0,5) :
        for j in range(0,5) :
            filecombine(files[i],files[j],opfiles[int(5*i+j)]+".hmm")

    return None

In [ ]:
!mkdir ctsModels
digit2model()
# model2digit = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
# model2digit = model2digit.astype("str")
# for i in range(20,25) :
#     files.download(model2digit[i]+".hmm")

In [ ]:
!ls
%cd /root/ctsModels/
#!cp *.hmm /root
!ls
%cd

In [70]:
def digit3model(files1=None,files2=None,folder="ctsModels") :
    if files1 is None :
        files1 = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
        files1 = files1.astype("str")
    if files2 is None :
        files2 = array([1,2,3,6,7])
        files2 = files2.astype("str")
        files = ["one.seq.hmm","two.seq.hmm","three.seq.hmm","six.seq.hmm","seven.seq.hmm"]
    opfiles = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
    opfiles = opfiles.astype("str")
    for i in range(0,25) :
        for j in range(0,5) :
            filecombine(files1[i]+".hmm",files[j],files1[i]+files2[j]+".hmm",Nstate="9")

    return None

In [ ]:
digit3model()

In [73]:
!zip -r /root/ctsModels.zip /root/ctsModels

files.download("/root/ctsModels.zip")

  adding: root/ctsModels/ (stored 0%)
  adding: root/ctsModels/613.hmm (deflated 77%)
  adding: root/ctsModels/12.hmm (deflated 77%)
  adding: root/ctsModels/137.hmm (deflated 78%)
  adding: root/ctsModels/662.hmm (deflated 83%)
  adding: root/ctsModels/766.hmm (deflated 83%)
  adding: root/ctsModels/162.hmm (deflated 78%)
  adding: root/ctsModels/313.hmm (deflated 83%)
  adding: root/ctsModels/61.hmm (deflated 76%)
  adding: root/ctsModels/672.hmm (deflated 77%)
  adding: root/ctsModels/716.hmm (deflated 77%)
  adding: root/ctsModels/767.hmm (deflated 82%)
  adding: root/ctsModels/76.hmm (deflated 75%)
  adding: root/ctsModels/627.hmm (deflated 77%)
  adding: root/ctsModels/176.hmm (deflated 78%)
  adding: root/ctsModels/316.hmm (deflated 77%)
  adding: root/ctsModels/317.hmm (deflated 78%)
  adding: root/ctsModels/617.hmm (deflated 78%)
  adding: root/ctsModels/621.hmm (deflated 78%)
  adding: root/ctsModels/731.hmm (deflated 77%)
  adding: root/ctsModels/362.hmm (deflated 77%)
  add

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
a = array([11,12,13])
a = a.astype("str")
a[1]=="12"

True

In [ ]:
#filecombine("one.seq.hmm","two.seq.hmm","check.txt")
filecombine("12.hmm","three.seq.hmm","123.hmm")

(12, 17) (6, 17)


In [ ]:
files.download("123.hmm")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [79]:
                                    #Give the file with sequences for cts. dev data
def ctsdevSeq(trainKM,fileWord="check.txt") :
    seq = []   
                                    #Go through required folders
    for subdir, dirs, files in os.walk("ctsHMM"):
        print(subdir)
        dirname = subdir.split(os.path.sep)[-1]
        print('Directory:', dirname)
        if dirname == "dev":
            for f in files:
                if f.endswith(".mfcc"):
                                    #Copy file (mfcc) to root and perform K-means
                    !cp -avr /root/ctsHMM/dev/$f /root
                    label = testKmeans(f,trainKM)
                    seq.append(label)
                    !rm $f

    seq = array(seq)
    
    seqDwnld(seq,fileWord)

    return None

In [80]:
                                    #Get the test sequence
ctsdevSeq(cenKM,"ctsHMM.dev")

ctsHMM
Directory: ctsHMM
ctsHMM/test
Directory: test
ctsHMM/dev
Directory: dev
'/root/ctsHMM/dev/262.mfcc' -> '/root/262.mfcc'
'/root/ctsHMM/dev/662.mfcc' -> '/root/662.mfcc'
'/root/ctsHMM/dev/66.mfcc' -> '/root/66.mfcc'
'/root/ctsHMM/dev/711.mfcc' -> '/root/711.mfcc'
'/root/ctsHMM/dev/12.mfcc' -> '/root/12.mfcc'
'/root/ctsHMM/dev/233.mfcc' -> '/root/233.mfcc'
'/root/ctsHMM/dev/217.mfcc' -> '/root/217.mfcc'
'/root/ctsHMM/dev/121.mfcc' -> '/root/121.mfcc'
'/root/ctsHMM/dev/622.mfcc' -> '/root/622.mfcc'
'/root/ctsHMM/dev/61.mfcc' -> '/root/61.mfcc'
'/root/ctsHMM/dev/11.mfcc' -> '/root/11.mfcc'
'/root/ctsHMM/dev/13.mfcc' -> '/root/13.mfcc'
'/root/ctsHMM/dev/122.mfcc' -> '/root/122.mfcc'
'/root/ctsHMM/dev/22.mfcc' -> '/root/22.mfcc'
'/root/ctsHMM/dev/111.mfcc' -> '/root/111.mfcc'
'/root/ctsHMM/dev/77.mfcc' -> '/root/77.mfcc'
'/root/ctsHMM/dev/277.mfcc' -> '/root/277.mfcc'
'/root/ctsHMM/dev/212.mfcc' -> '/root/212.mfcc'
'/root/ctsHMM/dev/16.mfcc' -> '/root/16.mfcc'
'/root/ctsHMM/dev/331.mfc

In [81]:
                                    #Download the test sequence files
files.download("ctsHMM.dev")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [82]:
!ls
%cd ctsHMM/dev
!ls
%cd

1	22.hmm	36.hmm	7	    ctsModels	   one.seq	  two.seq
11.hmm	23.hmm	37.hmm	71.hmm	    ctsModels.zip  one.seq.hmm	  two.seq.hmm
12.hmm	26.hmm	6	72.hmm	    DHMM.test	   seven.seq
13.hmm	27.hmm	61.hmm	73.hmm	    fp1.txt	   seven.seq.hmm
16.hmm	3	62.hmm	76.hmm	    fp2.txt	   six.seq
17.hmm	31.hmm	63.hmm	77.hmm	    fp3.txt	   six.seq.hmm
2	32.hmm	66.hmm	ctsHMM	    fp6.txt	   three.seq
21.hmm	33.hmm	67.hmm	ctsHMM.dev  fp7.txt	   three.seq.hmm
/root/ctsHMM/dev
111.mfcc  122.mfcc  16.mfcc   21.mfcc	262.mfcc  322.mfcc  662.mfcc  77.mfcc
116.mfcc  12.mfcc   172.mfcc  22.mfcc	277.mfcc  331.mfcc  66.mfcc
11.mfcc   132.mfcc  212.mfcc  233.mfcc	311.mfcc  61.mfcc   711.mfcc
121.mfcc  13.mfcc   217.mfcc  23.mfcc	31.mfcc   622.mfcc  71.mfcc
/root


In [ ]:
a = array([11,12,13,16,17,21,22,23,26,27,31,32,33,36,37,61,62,63,66,67,71,72,73,76,77])
b = array([1,2,3,6,7])
a = a.astype("str")
b = b.astype("str")
c = np.zeros(125)
for i in range(0,25) :
    for j in range(0,5) :
        c[int(5*i+j)] = a[i]+b[j]

c = c.astype("int")
c = c.astype("str")
c = list(c)
a = list(a)
d = a+c
print(c)
print(a)
print(d)

['111', '112', '113', '116', '117', '121', '122', '123', '126', '127', '131', '132', '133', '136', '137', '161', '162', '163', '166', '167', '171', '172', '173', '176', '177', '211', '212', '213', '216', '217', '221', '222', '223', '226', '227', '231', '232', '233', '236', '237', '261', '262', '263', '266', '267', '271', '272', '273', '276', '277', '311', '312', '313', '316', '317', '321', '322', '323', '326', '327', '331', '332', '333', '336', '337', '361', '362', '363', '366', '367', '371', '372', '373', '376', '377', '611', '612', '613', '616', '617', '621', '622', '623', '626', '627', '631', '632', '633', '636', '637', '661', '662', '663', '666', '667', '671', '672', '673', '676', '677', '711', '712', '713', '716', '717', '721', '722', '723', '726', '727', '731', '732', '733', '736', '737', '761', '762', '763', '766', '767', '771', '772', '773', '776', '777']
['11', '12', '13', '16', '17', '21', '22', '23', '26', '27', '31', '32', '33', '36', '37', '61', '62', '63', '66', '67', '71